In [19]:
import matplotlib.pyplot as plt
from scipy.signal import stft
from scipy import signal
from tqdm import tqdm
import numpy as np
import pathlib
import pickle
import os

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import layers
from tensorflow import keras

EPOCH = 100
KERNEL_SIZE = 3
BATCH_SIZE = 128
IMAGE_HEIGHT = 512
IMAGE_WIDTH = 512

DATA_PATH = "./pickle_files/"
SAVE_PATH = "./spectrogram_plt/"

In [20]:
record_list = []
pickle_input = dict()
X, y = [], []

print("[INFO] Read records file from ", DATA_PATH)
with open(DATA_PATH + 'RECORDS') as f:
    record_lines = f.readlines()

for i in range(len(record_lines)):
    record_list.append(str(record_lines[i].strip()))

for i in tqdm(range(len(record_list))):
    temp_path = DATA_PATH + "mit" + record_list[i] + ".pkl"
    with open(temp_path, 'rb') as f:
        pickle_input = pickle.load(f)
        for i in range(len(pickle_input[0])):
            X.append(pickle_input[0][i])

        for i in range(len(pickle_input[1])):
            check_ann = pickle_input[1][i]
            temp_ann_list = list()
            if check_ann == "N":            # Normal
                temp_ann_list.append(0)

            elif check_ann == "S":          # Supra-ventricular
                temp_ann_list.append(1)

            elif check_ann == "V":          # Ventricular
                temp_ann_list.append(2)

            elif check_ann == "F":          # False alarm
                temp_ann_list.append(3)

            else:                           # Unclassed 
                temp_ann_list.append(4)
            y.append(temp_ann_list)

[INFO] Read records file from  ./pickle_files/


100%|██████████| 48/48 [00:01<00:00, 47.06it/s]


In [ ]:
# 경로에 폴더가 없으면 폴더 만들기
def createDirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")

In [5]:
# fs = 36e3
# N = 1e3
# amp = 2 * np.sqrt(2)
# time = np.arange(N) / float(fs)
# for record in  X:
#     f, t, Sxx = signal.spectrogram(record, fs)
#     plt.specgram(Sxx)

In [6]:
# transformed_signal = []
# dt = 0.001

# PATH = "./spectrogram_plt/"
# createDirectory(PATH)

# for cnt in tqdm(range(len(X))):   
#     plt.specgram(X[cnt], cmap='jet_r', mode='psd', sides='default')

#     SAVE_PATH = PATH + str(cnt) + "_" + ".png"
#     plt.interactive(False)
#     # print(cnt)
                
# print(transformed_signal)

In [ ]:
data_dir = pathlib.Path(DATA_PATH)
print(data_dir)

In [ ]:
train_ds = keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    label_mode='int',
    # label_mode='categorical',
    color_mode='rgb',
    seed=1234,
    image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size=BATCH_SIZE,
    shuffle=True
)

In [ ]:
val_ds = keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    label_mode='int',
    # label_mode='categorical',
    color_mode='rgb',
    seed=1234,
    image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size=BATCH_SIZE,
    shuffle=True
)

In [ ]:
print(train_ds.unique)
print(val_ds.unique)

class_names = train_ds.class_names
print(class_names)
plt.figure(figsize=(10, 10))

for images, labels in train_ds.take(1):
  for i in range(10):
    ax = plt.subplot(5, 5, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [11]:
input_size = (256, 256, 1)

models = keras.Sequential([
    # tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
    layers.Conv2D(8, kernel_size=(4, 4), activation='relu', input_shape=input_size),
    layers.MaxPool2D(pool_size=(2, 2)),

    layers.Conv2D(13, kernel_size=(2, 2), activation='relu', input_shape=input_size),
    layers.MaxPool2D(pool_size=(2, 2)),

    layers.Conv2D(13, kernel_size=(2, 2), activation='relu', input_shape=input_size),
    layers.MaxPool2D(pool_size=(2, 2)),

    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(512, activation='relu'),
    layers.Dense(5, activation='softmax')
])

In [13]:
models.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 253, 253, 8)       136       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 126, 126, 8)      0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 125, 125, 13)      429       
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 62, 62, 13)       0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 61, 61, 13)        689       
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 30, 30, 13)      

In [15]:
# 콜백 설정
outDir = './cheakpoint/lefms_model/' # 이 경로에 best 모델이 저장된다.
model_names = outDir + 'weights-{val_accuracy:.4f}.h5'

def get_callbacks(patience = 50):
    earlystop = EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=patience)
    model_checkpoint = ModelCheckpoint(model_names, monitor='val_accuracy', verbose=1, save_best_only=True, period = 1)

    callbacks = [model_checkpoint]
    return callbacks